# 工业蒸汽预测
- 背景介绍
  
  火力发电的基本原理是：燃料在燃烧时加热水生成蒸汽，蒸汽压力推动汽轮机旋转，然后汽轮机带动发电机旋转，产生电能。在这一系列的能量转化中，影响发电效率的核心是锅炉的燃烧效率，即燃料燃烧加热水产生高温高压蒸汽。锅炉的燃烧效率的影响因素很多，包括锅炉的可调参数，如燃烧给量，一二次风，引风，返料风，给水水量；以及锅炉的工况，比如锅炉床温、床压，炉膛温度、压力，过热器的温度等

- 相关描述
  
    经脱敏后的锅炉传感器采集的数据（采集频率是分钟级别），根据锅炉的工况，预测产生的蒸汽量。

- 数据说明
  
    数据分成训练数据（train.txt）和测试数据（test.txt），其中字段”V0”-“V37”，这38个字段是作为特征变量，”target”作为目标变量。选手利用训练数据训练出模型，预测测试数据的目标变量，排名结果依据预测结果的MSE（mean square error）。

- 结果评估
  
    预测结果以mean square error作为评判标准。

## 前言

什么是`MSE`?从名字上来看，`MSE`就是均方误差。它的计算公式很像方差的计算公式。我的理解是，因为像这类问题都是回归预测问题，从而又引出了一些新的问题？
- 为什么要划分训练集和测试集
- 既然是预测回归问题，那么为什么不使用最小二乘法去拟合所有的数据从而得到一条直线来预测呢

针对这两个问题，首先，划分训练集和测试集的一个很重要的目的就是去**评估所得到模型的正确性**。假设你知道所有的数据，并且这些数据永远也不会增加，那么我觉得为了简单起见，大可以把数据直接拟合，然后对于这些已有的数据，就可以很好的“预测”。但是，世界是处在变化之中的，信息无时无刻不在增加，对于那些**我们从来没有见过的数据**，我们又怎么知道拟合出来曲线的正确性呢？如果模型只能很好的预测已经有的数据的话，那么这个模型还算做预测模型吗？
所以一般来说，我们会把数据集分为两部分，一部分用来训练(假设是80%)，那么剩下来的数据(20%)就可以用来检验我们**训练的模型预测出来的数据与我们实际的数据之间的差距**。显然，如果预测出来的数据 $\hat{y_i}$ 越接近于真实值 $ y_i $，那么就说明我们的预测结果也就越好。为了描述这个好的程度，所以也就有了`MSE`这个约定。
`MSE`的计算公式还是很简单的，它的计算公式为：$$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2 $$

是不是很像方差的计算公式，只不过`MSE`计算的是实际值于预测值之间的误差的平均值，同样的，为了避免有负数，两者的结果加上了平方。显然，`MSE`越小，代表模型的预测效果也就越好。

# 1 数据探索

## 1.1 下载数据

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")
 
%matplotlib inline

In [12]:

# 下载需要用到的数据集
# 注意，需要事先安装wget工具，如果没有请自行安装
!wget http://tianchi-media.oss-cn-beijing.aliyuncs.com/DSW/Industrial_Steam_Forecast/zhengqi_test.txt
!wget http://tianchi-media.oss-cn-beijing.aliyuncs.com/DSW/Industrial_Steam_Forecast/zhengqi_train.txt


--2024-06-25 21:05:17--  http://tianchi-media.oss-cn-beijing.aliyuncs.com/DSW/Industrial_Steam_Forecast/zhengqi_test.txt
Resolving tianchi-media.oss-cn-beijing.aliyuncs.com (tianchi-media.oss-cn-beijing.aliyuncs.com)... 211.154.222.168
Connecting to tianchi-media.oss-cn-beijing.aliyuncs.com (tianchi-media.oss-cn-beijing.aliyuncs.com)|211.154.222.168|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 466959 (456K) [text/plain]
Saving to: 'zhengqi_test.txt'

     0K .......... .......... .......... .......... .......... 10% 9.20M 0s
    50K .......... .......... .......... .......... .......... 21% 5.38M 0s
   100K .......... .......... .......... .......... .......... 32% 7.88M 0s
   150K .......... .......... .......... .......... .......... 43% 74.5M 0s
   200K .......... .......... .......... .......... .......... 54% 5.94M 0s
   250K .......... .......... .......... .......... .......... 65% 21.6M 0s
   300K .......... .......... .......... .......... .........

In [3]:
# 使用Pandas库`read_csv()`函数进行数据读取，分割符为‘\t’
train_data_file = "./zhengqi_train.txt"
test_data_file =  "./zhengqi_test.txt"

# pandas
train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8') # 训练集
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8') # 测试集

## 1.2 数据类型的查看

In [4]:
#查看特征变量信息
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2888 entries, 0 to 2887
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V0      2888 non-null   float64
 1   V1      2888 non-null   float64
 2   V2      2888 non-null   float64
 3   V3      2888 non-null   float64
 4   V4      2888 non-null   float64
 5   V5      2888 non-null   float64
 6   V6      2888 non-null   float64
 7   V7      2888 non-null   float64
 8   V8      2888 non-null   float64
 9   V9      2888 non-null   float64
 10  V10     2888 non-null   float64
 11  V11     2888 non-null   float64
 12  V12     2888 non-null   float64
 13  V13     2888 non-null   float64
 14  V14     2888 non-null   float64
 15  V15     2888 non-null   float64
 16  V16     2888 non-null   float64
 17  V17     2888 non-null   float64
 18  V18     2888 non-null   float64
 19  V19     2888 non-null   float64
 20  V20     2888 non-null   float64
 21  V21     2888 non-null   float64
 22  

In [5]:
# 测试集的数据
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V0      1925 non-null   float64
 1   V1      1925 non-null   float64
 2   V2      1925 non-null   float64
 3   V3      1925 non-null   float64
 4   V4      1925 non-null   float64
 5   V5      1925 non-null   float64
 6   V6      1925 non-null   float64
 7   V7      1925 non-null   float64
 8   V8      1925 non-null   float64
 9   V9      1925 non-null   float64
 10  V10     1925 non-null   float64
 11  V11     1925 non-null   float64
 12  V12     1925 non-null   float64
 13  V13     1925 non-null   float64
 14  V14     1925 non-null   float64
 15  V15     1925 non-null   float64
 16  V16     1925 non-null   float64
 17  V17     1925 non-null   float64
 18  V18     1925 non-null   float64
 19  V19     1925 non-null   float64
 20  V20     1925 non-null   float64
 21  V21     1925 non-null   float64
 22  

这些数据的输出结果有点令人困惑，经过查询后可以这样理解。